In [ ]:
# 練習14-1 影像金字塔 - 手工下取樣（使用cv2.getGaussianKernel）
import cv2
import numpy as np

# 原始影像
img = np.array([
    [197, 25, 106, 156, 159, 12],
    [149, 40, 107, 5, 71, 36],
    [163, 198, 226, 223, 156, 250],
    [222, 37, 68, 193, 157, 111],
    [42, 72, 250, 41, 75, 18],
    [159, 69, 235, 81, 179, 180]
], dtype=np.uint8)

print("原始影像 (6x6):")
print(img)
print()

# 步驟1: 使用 cv2.getGaussianKernel 建立 5x5 高斯濾波器
kernelY = cv2.getGaussianKernel(5, 0)  # 垂直方向的1D高斯核
kernelX = np.transpose(kernelY)  # 水平方向（轉置）

print("kernelY (5x1) - 垂直方向:")
print(kernelY)
print("\nkernelX (1x5) - 水平方向:")
print(kernelX)
print()

# 建立 5x5 的2D kernel
kernelXY = np.zeros((5, 5), dtype=np.float64)
for i in range(5):
    for j in range(5):
        kernelXY[i, j] = kernelX[0, i] * kernelY[j, 0]

print("kernelXY (5x5) - 2D高斯核:")
print(kernelXY)
print(f"總和: {kernelXY.sum()}")
print()

# 步驟2: 邊界填充 (BORDER_REFLECT_101 鏡像填充)
padded = cv2.copyMakeBorder(img, 2, 2, 2, 2, cv2.BORDER_REFLECT_101)
print("填充後影像 (10x10):")
print(padded)
print()

# 步驟3: 手工進行高斯濾波
rows, cols = img.shape
filtered = np.zeros_like(img, dtype=np.float64)

for i in range(rows):
    for j in range(cols):
        region = padded[i:i+5, j:j+5]
        # 標準卷積：element-wise相乘再相加
        filtered[i, j] = np.sum(region * kernelXY)

print("高斯濾波後影像 (6x6) - 浮點數:")
print(filtered)
print()

# 步驟4: 下取樣 - 取索引 0,2,4 行列，並使用 np.round() 四捨五入
downsampled = np.round(filtered[0::2, 0::2]).astype(np.uint8)

print("下取樣結果 (3x3) - [0::2, 0::2]:")
print(downsampled)
print()



In [ ]:
# 練習14-1-2 影像金字塔 - 手工上取樣（Upsampling）
import cv2
import numpy as np

# 原始影像 (3x3) - 使用練習14-1的下取樣結果
img = np.array([
    [106, 99, 93],
    [132, 136, 136],
    [107, 133, 117]
], dtype=np.uint8)

print("原始影像 (3x3):")
print(img)
print()

# 步驟1: 建立上取樣的空白影像 (6x6)
# 上取樣：在每個像素之間插入零
rows, cols = img.shape
upsampled_size = (rows * 2, cols * 2)
upsampled = np.zeros(upsampled_size, dtype=np.float64)

# 將原始像素值放入偶數位置 [0::2, 0::2]
upsampled[0::2, 0::2] = img

print("步驟1 - 零填充後的影像 (6x6):")
print(upsampled.astype(np.uint8))
print()

# 步驟2: 建立高斯濾波器並乘以4
kernelY = cv2.getGaussianKernel(5, 0)  # 垂直方向的1D高斯核
kernelX = np.transpose(kernelY)  # 水平方向（轉置）

# 建立 5x5 的2D kernel 並乘以4
kernelXY = np.zeros((5, 5), dtype=np.float64)
for i in range(5):
    for j in range(5):
        kernelXY[i, j] = kernelX[0, i] * kernelY[j, 0]

# 上取樣時要乘以4來補償零填充
kernelXY_scaled = kernelXY * 4

print("高斯核 (5x5) × 4:")
print(kernelXY_scaled)
print(f"總和: {kernelXY_scaled.sum()}")
print()

# 步驟3: 邊界填充 (BORDER_REFLECT_101 鏡像填充)
padded = cv2.copyMakeBorder(upsampled, 2, 2, 2, 2, cv2.BORDER_REFLECT_101)
print("填充後影像 (10x10):")
print(padded.astype(np.uint8))
print()

# 步驟4: 手工進行高斯濾波
rows_up, cols_up = upsampled.shape
filtered = np.zeros_like(upsampled, dtype=np.float64)

for i in range(rows_up):
    for j in range(cols_up):
        region = padded[i:i+5, j:j+5]
        # 標準卷積：element-wise相乘再相加
        filtered[i, j] = np.sum(region * kernelXY_scaled)

print("高斯濾波後影像 (6x6) - 浮點數:")
print(filtered)
print()

# 步驟5: 使用 int(x + 0.5) 標準四捨五入並轉換為uint8
result = np.array([[int(val + 0.5) for val in row] for row in filtered], dtype=np.uint8)

print("上取樣結果 (6x6):")
print(result)
print()



In [ ]:
# 練習14-2 影像金字塔 - 向下上採樣 (使用 cv2.pyrDown 和 cv2.pyrUp)
import cv2
import numpy as np
import matplotlib.pyplot as plt

# 設定 matplotlib 字體以避免中文顯示警告
plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei', 'SimHei', 'Arial Unicode MS']
plt.rcParams['axes.unicode_minus'] = False  # 解決負號顯示問題

# 讀取 lena.bmp
img = cv2.imread('C:\\img\\lena.bmp', 0)

if img is None:
    print("❌ 無法讀取 lena.bmp，請確認檔案路徑")
else:
    print("原始影像 shape:", img.shape)
    
    # 向下採樣 3 次
    print("\n向下採樣過程:")
    print("="*60)
    
    dn1 = cv2.pyrDown(img)
    print(f"第1次向下採樣: {img.shape} -> {dn1.shape}")
    
    dn2 = cv2.pyrDown(dn1)
    print(f"第2次向下採樣: {dn1.shape} -> {dn2.shape}")
    
    dn3 = cv2.pyrDown(dn2)
    print(f"第3次向下採樣: {dn2.shape} -> {dn3.shape}")
    
    # 向上採樣 3 次
    print("\n向上採樣過程:")
    print("="*60)
    
    up1 = cv2.pyrUp(dn3)
    print(f"第1次向上採樣: {dn3.shape} -> {up1.shape}")
    
    up2 = cv2.pyrUp(up1)
    print(f"第2次向上採樣: {up1.shape} -> {up2.shape}")
    
    up3 = cv2.pyrUp(up2)
    print(f"第3次向上採樣: {up2.shape} -> {up3.shape}")
    
    # 檢查是否還原成功
    print("\n還原結果:")
    print("="*60)
    print(f"原始影像 shape: {img.shape}")
    print(f"還原影像 shape: {up3.shape}")
    
    if img.shape == up3.shape:
        print("✅ Shape 相同")
        
        # 檢查像素值是否完全相同
        if np.array_equal(img, up3):
            print("✅ 完全還原成功！像素值完全相同")
        else:
            # 計算差異
            diff = cv2.absdiff(img, up3)
            max_diff = np.max(diff)
            mean_diff = np.mean(diff)
            
            print("❌ 像素值不同")
            print(f"最大差異: {max_diff}")
            print(f"平均差異: {mean_diff:.4f}")
            print(f"差異標準差: {np.std(diff):.4f}")
            
            # 計算差異百分比
            non_zero_diff = np.count_nonzero(diff)
            total_pixels = img.shape[0] * img.shape[1]
            diff_percentage = (non_zero_diff / total_pixels) * 100
            print(f"有差異的像素比例: {diff_percentage:.2f}%")
    else:
        print("❌ Shape 不同，無法完全還原")
        print(f"差異: 原始 {img.shape} vs 還原 {up3.shape}")
    
    # 視覺化顯示
    print("\n視覺化結果:")
    print("="*60)
    
    plt.figure(figsize=(15, 10))
    
    # Original image
    plt.subplot(3, 3, 1)
    plt.imshow(img, cmap='gray')
    plt.title(f'Original\n{img.shape}')
    plt.axis('off')
    
    # Downsample process
    plt.subplot(3, 3, 2)
    plt.imshow(dn1, cmap='gray')
    plt.title(f'Downsample 1\n{dn1.shape}')
    plt.axis('off')
    
    plt.subplot(3, 3, 3)
    plt.imshow(dn2, cmap='gray')
    plt.title(f'Downsample 2\n{dn2.shape}')
    plt.axis('off')
    
    plt.subplot(3, 3, 4)
    plt.imshow(dn3, cmap='gray')
    plt.title(f'Downsample 3\n{dn3.shape}')
    plt.axis('off')
    
    # Upsample process
    plt.subplot(3, 3, 5)
    plt.imshow(up1, cmap='gray')
    plt.title(f'Upsample 1\n{up1.shape}')
    plt.axis('off')
    
    plt.subplot(3, 3, 6)
    plt.imshow(up2, cmap='gray')
    plt.title(f'Upsample 2\n{up2.shape}')
    plt.axis('off')
    
    plt.subplot(3, 3, 7)
    plt.imshow(up3, cmap='gray')
    plt.title(f'Reconstructed\n{up3.shape}')
    plt.axis('off')
    
    # Difference map
    if img.shape == up3.shape:
        plt.subplot(3, 3, 8)
        diff_visual = cv2.absdiff(img, up3)
        # Amplify difference for visualization
        diff_enhanced = cv2.convertScaleAbs(diff_visual, alpha=5)
        plt.imshow(diff_enhanced, cmap='hot')
        plt.title(f'Difference (x5)\nMax: {np.max(diff_visual)}')
        plt.axis('off')
        plt.colorbar()
        
        # Original vs Reconstructed comparison
        plt.subplot(3, 3, 9)
        # Left half: original, right half: reconstructed
        comparison = img.copy()
        mid = img.shape[1] // 2
        comparison[:, mid:] = up3[:, mid:]
        plt.imshow(comparison, cmap='gray')
        plt.title('Left: Original | Right: Reconstructed')
        plt.axis('off')
    
    plt.tight_layout()
    plt.show()
    


In [ ]:
import cv2
import numpy as np

# 讀取影像
img = cv2.imread('C:/img/lena.bmp', 0)

# === 向下採樣三次 ===
d1 = cv2.pyrDown(img)
d2 = cv2.pyrDown(d1)
d3 = cv2.pyrDown(d2)

# === 向上採樣三次 ===
u1 = cv2.pyrUp(d3)     # 對應 d2
u2 = cv2.pyrUp(u1)     # 對應 d1
u3 = cv2.pyrUp(u2)     # 對應 img

# === 計算差異 ===
diff1 = cv2.absdiff(d2, u1)
diff2 = cv2.absdiff(d1, u2)
diff3 = cv2.absdiff(img, u3)

# === 顯示 ===
cv2.imshow("Diff 1 (d2 vs pyrUp(d3))", diff1)
cv2.imshow("Diff 2 (d1 vs pyrUp(u1))", diff2)
cv2.imshow("Diff 3 (img vs pyrUp(u2))", diff3)

cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
# 練習14-4 拉普拉斯金字塔還原
import cv2
import numpy as np
import matplotlib.pyplot as plt

# 設定 matplotlib 字體
plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei', 'SimHei', 'Arial Unicode MS']
plt.rcParams['axes.unicode_minus'] = False

# 讀取影像
img = cv2.imread('C:/img/lena.bmp', 0)

if img is None:
    print("❌ 無法讀取 lena.bmp")
else:
    print("拉普拉斯金字塔還原過程")
    print("="*80)
    print(f"原始影像 shape: {img.shape}")
    
    # === 步驟1: 向下採樣 ===
    down = cv2.pyrDown(img)
    print(f"\n步驟1 - 向下採樣: {img.shape} -> {down.shape}")
    
    # === 步驟2: 向上採樣 (Up) ===
    up = cv2.pyrUp(down)
    print(f"步驟2 - 向上採樣: {down.shape} -> {up.shape}")
    
    # === 步驟3: 計算拉普拉斯影像 (L = 原圖 - Up) ===
    # 使用 int16 避免 uint8 的溢位問題
    L = cv2.subtract(img.astype(np.int16), up.astype(np.int16))
    print(f"\n步驟3 - 拉普拉斯影像 L = 原圖 - Up")
    print(f"  L 的範圍: [{np.min(L)}, {np.max(L)}]")
    print(f"  L 的平均值: {np.mean(L):.4f}")
    print(f"  L 的標準差: {np.std(L):.4f}")
    
    # === 步驟4: 拉普拉斯還原 (還原 = L + Up) ===
    restored = cv2.add(L.astype(np.int16), up.astype(np.int16))
    restored = np.clip(restored, 0, 255).astype(np.uint8)
    print(f"\n步驟4 - 拉普拉斯還原: 還原 = L + Up")
    
    # === 驗證還原是否成功 ===
    print("\n" + "="*80)
    print("驗證還原結果:")
    print("-" * 80)
    
    if np.array_equal(img, restored):
        print("✅ 完全還原成功！像素值完全相同")
    else:
        diff = cv2.absdiff(img, restored)
        max_diff = np.max(diff)
        mean_diff = np.mean(diff)
        non_zero = np.count_nonzero(diff)
        
        print(f"❌ 還原有微小差異（可能由於數值運算誤差）")
        print(f"  最大差異: {max_diff}")
        print(f"  平均差異: {mean_diff:.6f}")
        print(f"  有差異的像素數: {non_zero}")
        
        if max_diff <= 1:
            print("  ✅ 差異在容許範圍內（≤1），視為成功還原")
    
    print("="*80)
    
    # === 視覺化 ===
    fig = plt.figure(figsize=(15, 4))
    
    # 第三行：還原結果和驗證
    plt.subplot(1, 3, 1)
    plt.imshow(restored, cmap='gray')
    plt.title(f'Restored (L + Up)\n{restored.shape}', fontsize=12)
    plt.axis('off')
    
    plt.subplot(1, 3, 2)
    plt.imshow(img, cmap='gray')
    plt.title('Original (for comparison)', fontsize=12)
    plt.axis('off')
    
    # 還原差異
    restore_diff = cv2.absdiff(img, restored)
    plt.subplot(1, 3, 3)
    plt.imshow(restore_diff, cmap='gray')
    plt.title(f'Original - Restored\nMax: {np.max(restore_diff)}', fontsize=12)
    plt.axis('off')
    
    plt.tight_layout()
    plt.show()
    
 
